# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from joblib import dump

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import metrics

In [2]:
df = pd.read_csv("data/cleaned_KPIs.csv")
df.head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,0.001761,0,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,0.002468,1,0
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,0.003077,2,0
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,0.001721,3,0
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,0.002213,4,0


In [3]:
df = df.drop(['cell_id','index'],axis=1)

# 1. Isolation Forest

In [4]:
IF=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1,random_state=42)

In [5]:
IF.fit(df)
# score_samples = - score  
IF_score = -1 * IF.score_samples(df)
IF_score[:20]

array([0.51295825, 0.48509757, 0.53648751, 0.50925568, 0.48977436,
       0.52157804, 0.50460632, 0.47403398, 0.47869537, 0.4786467 ,
       0.46769137, 0.46808774, 0.47676634, 0.48726571, 0.48067503,
       0.50214502, 0.47844321, 0.47284323, 0.46897991, 0.46007395])

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


## 2.1 Build model

**(1) Standardize** 

In [6]:
df_scaled = StandardScaler().fit_transform(df)

**(2) Build Encoder**

In [7]:
original_dim = df.shape[1]
intermediate_dim = 16
latent_dim = 4

# encoder input
inputs = keras.Input(shape=(original_dim,))
x = layers.Dense(intermediate_dim, activation="relu")(inputs)
z = layers.Dense(latent_dim, activation="relu")(x)

# Create encoder
encoder = keras.Model(inputs, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 68        
Total params: 404
Trainable params: 404
Non-trainable params: 0
_________________________________________________________________


**(3) Build Decoder**

In [8]:
# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
y = layers.Dense(original_dim, activation='relu')(x)
decoder = keras.Model(latent_inputs, y, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               [(None, 4)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_3 (Dense)              (None, 20)                340       
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________


**(4) Build AutoEncoder**

In [15]:
# Instantiate VAE model
outputs = decoder(encoder(inputs))
ae = keras.Model(inputs, outputs, name='auto-encoder')

# Optimiser 
adam = keras.optimizers.Adam(learning_rate=0.001)

# Add loss function
ae.compile(optimizer=adam,loss="mse")


Notices:
- weight of kl_loss?  [theoretical insight](https://stats.stackexchange.com/questions/332179/how-to-weight-kld-loss-vs-reconstruction-loss-in-variational-auto-encoder) + [a more specific example](https://stats.stackexchange.com/questions/341954/balancing-reconstruction-vs-kl-loss-variational-autoencoder)
- reconsctruction_loss : if use `binary_crossentropy`( $BCE(y,p)=-y.log(p)-(1-y).log(1-p)$ ), the input value should be in 0~1. In our case, since we use 'StandardScaler', we'd better use `mse`, otherwise, when training the model, the loss becomes -inf 

**(5) Train model**

In [16]:
epochs = 500
batch_size = 128
callback = EarlyStopping(monitor='val_loss',patience=10)

# Train model
ae.fit(df_scaled, df_scaled,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(df_scaled, df_scaled),
        shuffle=True,
#         verbose=False,
        callbacks=[callback]
       )

Train on 8279 samples, validate on 8279 samples
Epoch 1/500
8279/8279 [==============================] - 0s 45us/sample - loss: 0.8233 - val_loss: 0.8030
Epoch 2/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7958 - val_loss: 0.7892
Epoch 3/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7852 - val_loss: 0.7812
Epoch 4/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7783 - val_loss: 0.7751
Epoch 5/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7728 - val_loss: 0.7695
Epoch 6/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7671 - val_loss: 0.7641
Epoch 7/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7628 - val_loss: 0.7609
Epoch 8/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7599 - val_loss: 0.7585
Epoch 9/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7578 - val_loss: 0.756

8279/8279 [==============================] - 0s 16us/sample - loss: 0.7371 - val_loss: 0.7370
Epoch 78/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7370 - val_loss: 0.7369
Epoch 79/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7370 - val_loss: 0.7369
Epoch 80/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7369 - val_loss: 0.7368
Epoch 81/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7369 - val_loss: 0.7369
Epoch 82/500
8279/8279 [==============================] - 0s 17us/sample - loss: 0.7368 - val_loss: 0.7368
Epoch 83/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7368 - val_loss: 0.7367
Epoch 84/500
8279/8279 [==============================] - 0s 15us/sample - loss: 0.7367 - val_loss: 0.7366
Epoch 85/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7367 - val_loss: 0.7366
Epoch 86/500
8279/8279 [==========================

8279/8279 [==============================] - 0s 15us/sample - loss: 0.7350 - val_loss: 0.7349
Epoch 154/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 155/500
8279/8279 [==============================] - 0s 20us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 156/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 157/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 158/500
8279/8279 [==============================] - 0s 16us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 159/500
8279/8279 [==============================] - 0s 17us/sample - loss: 0.7349 - val_loss: 0.7348
Epoch 160/500
8279/8279 [==============================] - 0s 17us/sample - loss: 0.7348 - val_loss: 0.7348
Epoch 161/500
8279/8279 [==============================] - 0s 18us/sample - loss: 0.7348 - val_loss: 0.7349
Epoch 162/500
8279/8279 [=================

Epoch 229/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7342 - val_loss: 0.7342
Epoch 230/500
8279/8279 [==============================] - 0s 18us/sample - loss: 0.7342 - val_loss: 0.7341
Epoch 231/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7342 - val_loss: 0.7342
Epoch 232/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7342 - val_loss: 0.7342
Epoch 233/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7342 - val_loss: 0.7342
Epoch 234/500
8279/8279 [==============================] - 0s 19us/sample - loss: 0.7342 - val_loss: 0.7342
Epoch 235/500
8279/8279 [==============================] - 0s 23us/sample - loss: 0.7342 - val_loss: 0.7341
Epoch 236/500
8279/8279 [==============================] - 0s 24us/sample - loss: 0.7342 - val_loss: 0.7341
Epoch 237/500
8279/8279 [==============================] - 0s 26us/sample - loss: 0.7342 - val_loss: 0.7341
Epoch 238/500
8279/8279 [===

## 2.2 [Detect Anomaly](https://towardsdatascience.com/hands-on-anomaly-detection-with-variational-autoencoders-d4044672acd5)
Detect anomaly by finding those who have a high reconstruction loss.


Steps:
1. Measure error between the original train (clean/normal) set and the output of the model, and generate an error vector representing the error term of each sample.

2. Find a relatively extreme value on that vector to use as your error threshold.

3. Run the model over the test or real data, in which anomalies are probably mixed with normal data.

4. Measure the reconstruction error and mark samples that exhibit an error term higher than the error threshold as anomalies.

In [17]:
def mse(original,reconstructed):
    return np.mean((original-reconstructed)**2,axis=1)

In [19]:
from sklearn.preprocessing import MinMaxScaler
pred  = ae.predict(df_scaled)

AE_loss = mse(df_scaled,pred)
AE_loss[:20]

array([0.93303628, 0.63390621, 1.18056078, 0.396229  , 0.35391262,
       1.20375977, 5.06754622, 1.02759443, 1.45984249, 2.16254389,
       0.91075599, 0.37151143, 0.6010824 , 1.40124381, 0.94254187,
       1.07563632, 1.13370866, 0.45573532, 0.70301635, 0.77639674])

# 3. AE + IF (stacking)

In [21]:
encoder_output = encoder.predict(df_scaled)
df_encoded = pd.DataFrame(encoder_output)
df_encoded

,0,1,2,3
0,2.784741,0.206964,1.077084,3.274514
1,3.100837,0.569643,0.070241,4.819953
2,4.234570,0.199034,0.448586,6.097384
3,2.354728,0.279683,0.316227,3.145494
4,2.812106,0.626090,0.000000,4.275085
...,...,...,...,...
8274,6.689503,1.460762,3.779755,0.483200
8275,7.058461,1.285158,3.801746,0.982202
8276,4.837620,3.414821,4.729827,2.103478
8277,5.475404,1.003208,2.551054,0.299206


In [23]:
AE_IF=IsolationForest(n_estimators=150, 
                       max_samples ='auto', 
                       max_features=1,random_state=42)
AE_IF.fit(df_encoded)

IsolationForest(max_features=1, n_estimators=150, random_state=42)

In [24]:
AE_IF_score = -1 * AE_IF.score_samples(df_encoded)
AE_IF_score[:20]

array([0.53758229, 0.54292233, 0.56003275, 0.51416726, 0.53251386,
       0.54095954, 0.5184242 , 0.49401429, 0.53345771, 0.49237882,
       0.48762808, 0.51501159, 0.5295732 , 0.50430891, 0.50259002,
       0.51355859, 0.53719554, 0.51235446, 0.54929128, 0.50592772])

# 4. Combination of results

In [25]:
df['IF_score'] = IF_score
df['IF_score'].describe()

count    8279.000000
mean        0.451213
std         0.030114
min         0.403969
25%         0.428277
50%         0.443766
75%         0.471093
max         0.599004
Name: IF_score, dtype: float64

In [26]:
df['AE_loss'] = AE_loss
df['AE_loss'].describe()

count    8279.000000
mean        0.734103
std         2.080939
min         0.027187
25%         0.211260
50%         0.376910
75%         0.767105
max        97.651455
Name: AE_loss, dtype: float64

Notice there is some extreme value in `VAE_loss`, which may affect our later visualization 

In [27]:
df['AE_loss'].quantile(0.99)

4.788723220525283

In [28]:
df['STACKING_score'] = AE_IF_score
df['STACKING_score'].describe()

count    8279.000000
mean        0.484142
std         0.034707
min         0.423684
25%         0.454220
50%         0.483750
75%         0.507313
max         0.705713
Name: STACKING_score, dtype: float64

In [29]:
df.head()

,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,CELL_AVAILABILITY_RATIO,RACH_Stp_Completion_SR_RATIO,Inter_RAT_HO_SR_UTRAN_SRVCC_RATIO,E_RAB_QCI1_DR_RATIO,...,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,E_RAB_DR_RATIO,HOUR,cell,IF_score,AE_loss,STACKING_score
0,3.779737e+10,3.947172e+09,15.0,4727.0,0.809859,0.992427,1.0,0.962688,0.576414,0.000000,...,0.770642,0.963636,0.963636,0.400000,0.001761,0,0,0.512958,0.933036,0.537582
1,3.684898e+10,4.088752e+09,6.0,3076.0,0.886792,0.993288,1.0,0.973207,0.576414,0.000000,...,0.842105,1.000000,1.000000,0.500000,0.002468,1,0,0.485098,0.633906,0.542922
2,3.292677e+10,5.016897e+09,8.0,3501.0,0.938356,0.994664,1.0,0.966330,0.000000,0.013889,...,0.931624,1.000000,0.966667,0.375000,0.003077,2,0,0.536488,1.180561,0.560033
3,3.021547e+10,5.139107e+09,9.0,2275.0,0.860215,0.994819,1.0,0.943216,0.576414,0.000000,...,0.816901,0.947368,0.947368,0.777778,0.001721,3,0,0.509256,0.396229,0.514167
4,3.082176e+10,4.250716e+09,17.0,2178.0,0.840426,0.995952,1.0,0.936256,0.576414,0.000000,...,0.794521,1.000000,1.000000,0.764706,0.002213,4,0,0.489774,0.353913,0.532514


# 5. Save Results

## 5.1 Save models

In [32]:
#save model
dump(IF,"model/IF.joblib")

['model/IF.joblib']

In [35]:
ae.save("model\AE")
encoder.save("model\Encoder")
decoder.save("model\Decoder")

INFO:tensorflow:Assets written to: model\AE\assets
INFO:tensorflow:Assets written to: model\Encoder\assets
INFO:tensorflow:Assets written to: model\Decoder\assets


In [37]:
#save model
dump(AE_IF,"model/AE_IF_stacking.joblib")

['model/AE_IF_stacking.joblib']

## 5.2 Save Data

In [38]:
df.to_csv("data/KPIs_with_metrics.csv",index=False)